In [1]:
import serial

# Replace 'XXXX' with the actual identifier for your device
device_name = "/dev/tty.usbserial-AQ0458B0"
baudrate = 9600

try:
    # Attempt to establish a serial connection
    serial_object = serial.Serial(port=device_name, baudrate=baudrate)
    print(f"Connected to {device_name}")
except serial.SerialException as e:
    # If there is an error, it will be printed here
    print(f"Error connecting to {device_name}: {e}")


Connected to /dev/tty.usbserial-AQ0458B0


In [16]:
import serial
import time

class ColdPlateSlimDriver:
    def __init__(self, temp_mode_number, max_temp_lag=0, heating_rate_deg_per_min=100, cooling_rate_deg_per_min=100):
        self.serial_number = "29517"
        self.device_name = "/dev/tty.usbserial-AQ0458B0"  # Updated to the confirmed port on your MacBook
        self.baudrate = 9600
        self.bytesize = serial.EIGHTBITS
        self.parity = serial.PARITY_NONE
        self.stopbits = serial.STOPBITS_ONE
        self.read_timeout = 2
        self.write_timeout = 2
        self.height = 45
        self.temp = 0
        self.deck_position = 7  # Update if needed for your setup
        self.max_temp_lag = max_temp_lag
        self.heating_rate_deg_per_min = heating_rate_deg_per_min
        self.cooling_rate_deg_per_min = cooling_rate_deg_per_min

        # Initialize the serial connection to the thermal module
        try:
            self.serial_object = serial.Serial(
                port=self.device_name,
                baudrate=self.baudrate,
                bytesize=self.bytesize,
                parity=self.parity,
                stopbits=self.stopbits,
                timeout=self.read_timeout,
                write_timeout=self.write_timeout
            )
        except serial.SerialException as e:
            print(f"Could not open serial port {self.device_name}: {e}")
            self.serial_object = None  # This will handle the case where the serial connection could not be made
    def _reset_buffers(self):
        """
        Clears the input and output buffers, ensuring there's no leftover data.
        """
        if self.serial_object is None:
            print("Serial connection not established. Cannot reset buffers.")
            return

        self.serial_object.reset_input_buffer()
        self.serial_object.reset_output_buffer()

    def _read_response(self):
        """
        Reads all data from the serial buffer until there's no more left.
        """
        if self.serial_object is None:
            return "Dummy response - no serial connection."

        output_lines = self.serial_object.readlines()
        output_string = "".join(l.decode("utf-8") for l in output_lines)
        return output_string

    def _send_command(self, my_command):
        """
        Sends a command to the thermal module and waits for the response.
        """
        SERIAL_ACK = "\r\n"

        if self.serial_object is None:
            print(f"Dummy command sent: {my_command}")
            return "Dummy response - no serial connection."

        command = my_command + SERIAL_ACK
        self.serial_object.write(command.encode())
        self.serial_object.flush()
        return self._read_response()

    def get_info(self):
        """
        Requests info from the thermal module. The exact command may need to be updated.
        """
        if self.serial_object is None:
            return "Dummy info - no serial connection."

        return self._send_command("info")

    def get_temp(self):
        """
        Requests the current temperature. The exact command may need to be updated.
        """
        if self.serial_object is None:
            return self.temp  # Returns the last set temperature if no connection.

        response = self._send_command("getTempActual")
        try:
            return float(response)
        except ValueError:
            print(f"Received non-numeric response: {response}")
            return None  # or some error value that makes sense for your application

    def set_temp(self, my_temp):
        """
        Sets the target temperature on the thermal module. The exact command may need to be updated.
        """
        if self.serial_object is None:
            self.temp = my_temp  # Updates the last set temperature if no connection.
            print("Dummy set temperature - no serial connection.")
            return

        temp = int(float(my_temp) * 10)  # Assumes the temperature must be sent as an integer multiplied by 10.
        self._send_command(f'setTempTarget{temp:03}')
        self._send_command("tempOn")

    def quick_temp(self, temp_target, overshot=10):
        #if temp_target > 60 or temp_target < 0:
        #    raise Exception("This function currently only works for temps between 0°C and 60°C")
        
        start_temp = self.get_temp()

        # Calculate the overshot temperature based on the direction of the temperature change
        if temp_target > start_temp:
            overshot_temp = temp_target + overshot
        else:
            overshot_temp = temp_target - overshot

        # Estimate delay time for the temperature to change based on a rate of change
        delay_seconds = abs(temp_target - start_temp) * (60 / 7)
        
        # Set the temperature to the overshot temperature first
        self.set_temp(overshot_temp)
        time.sleep(delay_seconds)
        
        # Then set it to the target temperature
        self.set_temp(temp_target)

    def temp_off(self):
        # Turns off the temperature control
        if self.serial_object is None:
            self.temp = 25  # Dummy value if no connection is established
            print("Dummy temp off - no serial connection.")
        else:
            self._send_command("tempOff")  # Replace with your thermal module's appropriate command

    def deactivate(self):
        # A convenience method that delegates to temp_off
        self.temp_off()

    def __del__(self):
        # Ensures the temperature is turned off and the serial connection is closed when the object is deleted
        try:
            self.temp_off()
            if self.serial_object is not None:
                self.serial_object.close()
        except Exception as e:
            print(f"Error in destructor: {e}")



In [26]:
import serial
import time

class ColdPlateSlimDriver:
    def __init__(self, temp_mode_number, max_temp_lag=0, heating_rate_deg_per_min=100, cooling_rate_deg_per_min=100):
        self.serial_number = "29517"
        self.device_name = "/dev/tty.usbserial-AQ0458B0"  # Updated to the confirmed port on your MacBook
        self.baudrate = 9600
        self.bytesize = serial.EIGHTBITS
        self.parity = serial.PARITY_NONE
        self.stopbits = serial.STOPBITS_ONE
        self.read_timeout = 2
        self.write_timeout = 2
        self.height = 45
        self.temp = 0
        self.deck_position = 7  # Update if needed for your setup
        self.max_temp_lag = max_temp_lag
        self.heating_rate_deg_per_min = heating_rate_deg_per_min
        self.cooling_rate_deg_per_min = cooling_rate_deg_per_min

        # Initialize the serial connection to the thermal module
        try:
            self.serial_object = serial.Serial(
                port=self.device_name,
                baudrate=self.baudrate,
                bytesize=self.bytesize,
                parity=self.parity,
                stopbits=self.stopbits,
                timeout=self.read_timeout,
                write_timeout=self.write_timeout
            )
        except serial.SerialException as e:
            print(f"Could not open serial port {self.device_name}: {e}")
            self.serial_object = None  # This will handle the case where the serial connection could not be made
    def _reset_buffers(self):
        """
        Clears the input and output buffers, ensuring there's no leftover data.
        """
        if self.serial_object is None:
            print("Serial connection not established. Cannot reset buffers.")
            return

        self.serial_object.reset_input_buffer()
        self.serial_object.reset_output_buffer()

    def _read_response(self):
        """
        Reads all data from the serial buffer until there's no more left.
        """
        if self.serial_object is None:
            return "Dummy response - no serial connection."

        output_lines = self.serial_object.readlines()
        output_string = "".join(l.decode("utf-8") for l in output_lines)
        return output_string

    def _send_command(self, my_command):
        """
        Sends a command to the thermal module and waits for the response.
        """
        SERIAL_ACK = "\r\n"

        if self.serial_object is None:
            print(f"Dummy command sent: {my_command}")
            return "Dummy response - no serial connection."

        command = my_command + SERIAL_ACK
        self.serial_object.write(command.encode())
        self.serial_object.flush()
        return self._read_response()

    def get_info(self):
        """
        Requests info from the thermal module. The exact command may need to be updated.
        """
        if self.serial_object is None:
            return "Dummy info - no serial connection."

        return self._send_command("info")

    def get_temp(self):
        """
        Requests the current temperature. The exact command may need to be updated.
        """
        if self.serial_object is None:
            return self.temp  # Returns the last set temperature if no connection.

        response = self._send_command("getTempActual")
        try:
            return float(response)
        except ValueError:
            print(f"Received non-numeric response: {response}")
            return None  # or some error value that makes sense for your application

    def set_temp(self, my_temp):
        """
        Sets the target temperature on the thermal module. The exact command may need to be updated.
        """
        if self.serial_object is None:
            self.temp = my_temp  # Updates the last set temperature if no connection.
            print("Dummy set temperature - no serial connection.")
            return

        temp = int(float(my_temp) * 10)  # Assumes the temperature must be sent as an integer multiplied by 10.
        self._send_command(f'setTempTarget{temp:03}')
        self._send_command("tempOn")
    
    
    def temp_off(self):
        # Turns off the temperature control
        if self.serial_object is None:
            self.temp = 25  # Dummy value if no connection is established
            print("Dummy temp off - no serial connection.")
        else:
            self._send_command("tempOff")  # Replace with your thermal module's appropriate command

    def deactivate(self):
        # A convenience method that delegates to temp_off
        self.temp_off()

    def __del__(self):
        # Ensures the temperature is turned off and the serial connection is closed when the object is deleted
        try:
            self.temp_off()
            if self.serial_object is not None:
                self.serial_object.close()
        except Exception as e:
            print(f"Error in destructor: {e}")

    def quick_temp(self, temp_target, overshot=10):

        start_temp = self.get_temp()
        end_time = 120 * 60  # 2 hours in seconds

        # Calculate the overshot temperature based on the direction of the temperature change
        if temp_target > start_temp:
            overshot_temp = temp_target + overshot
        else:
            overshot_temp = temp_target - overshot

        # Set the temperature to the overshot temperature first
        self.set_temp(overshot_temp)

        # Initialize the elapsed time
        elapsed = 0
        interval = 10  # Check every 10 seconds

        # Monitoring loop with auto-shutoff after 2 hours
        while elapsed < end_time:
            current_temp = self.get_temp()
            print(f"time elapsed (min): \t{elapsed / 60:.1f}")
            print(f"temp_mod       (C): \t{current_temp}")

            # Check if we've reached or passed the target temperature
            if (temp_target > start_temp and current_temp >= temp_target) or \
               (temp_target < start_temp and current_temp <= temp_target):
                break

            time.sleep(interval)
            elapsed += interval

        # After overshooting, set to the target temperature
        self.set_temp(temp_target)

        # Continue monitoring until 2 hours have elapsed
        while elapsed < end_time:
            current_temp = self.get_temp()
            print(f"time elapsed (min): \t{elapsed / 60:.1f}")
            print(f"temp_mod       (C): \t{current_temp}")
            time.sleep(interval)
            elapsed += interval

        # Auto-shutoff after 2 hours
        if elapsed >= end_time:
            self.temp_off()
            print("Auto-shutoff triggered after 2 hours.")


In [27]:
# Instantiate the ColdPlateSlimDriver class
temp_mod = ColdPlateSlimDriver(0)

In [23]:
# check if it works
temp_mod.set_temp(10.0)

In [24]:
temp_mod._send_command("tempOff")

'ok\r\n'

In [ ]:
temp_mod.quick_temp(90.0)

time elapsed (min): 	0.0
temp_mod       (C): 	66.852
time elapsed (min): 	0.2
temp_mod       (C): 	72.672
time elapsed (min): 	0.3
temp_mod       (C): 	77.08
time elapsed (min): 	0.5
temp_mod       (C): 	80.527
time elapsed (min): 	0.7
temp_mod       (C): 	83.332
time elapsed (min): 	0.8
temp_mod       (C): 	85.641
time elapsed (min): 	1.0
temp_mod       (C): 	87.557
time elapsed (min): 	1.2
temp_mod       (C): 	89.16
time elapsed (min): 	1.3
temp_mod       (C): 	90.571


In [28]:
temp_mod.quick_temp(90.0)
#temp_mod_2.set_temp(80.0)

#stop the execution of this cell when the sample temp stabilizes

elapsed = 0
for i in range(2400):
    #print(f"time elapsed : {elapsed} min \t temp_mod: {temp_mod.get_temp()}")
    print(f"time elapsed (min): \t{elapsed}")
    print(f"temp_mod       (C): \t{temp_mod.get_temp()}")
    time.sleep(10)
    elapsed+=0.1

KeyboardInterrupt: 

In [15]:
temp_mod.set_temp(90.0)

#stop the execution of this cell when the sample temp stabilizes

elapsed = 0
for i in range(2400):
    #print(f"time elapsed : {elapsed} min \t temp_mod: {temp_mod.get_temp()}")
    print(f"time elapsed (min): \t{elapsed}")
    print(f"temp_mod       (C): \t{temp_mod.get_temp()}")
    
    time.sleep(10)
    elapsed+=0.1

time elapsed (min): 	0
temp_mod       (C): 	59.021


KeyboardInterrupt: 

In [ ]:
# Send a direct command to get the actual temperature
actual_temp = temp_mod._send_command("getTemp4ActualValue")

NameError: name 'temp_mod' is not defined

In [ ]:
# Send a direct command to set the actual temperature to 79.1°C (assuming your device uses such a command)
temp_mod._send_command("setTemp4ActualValueXXX")